
# Bokeh for Time Series Analysis
<hr style="border: 2px solid black;">


<img src="./images/bokeh.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">

<img src="./images/bokeh_at_ag_glance.png" alt="bokeh Logo" width="1000"/>
<hr style="border: 2px solid black;">
**Introduction to Bokeh**
Bokeh is an interactive visualization library for Python that targets modern web browsers for presentation.
Unlike Matplotlib, which is primarily designed for static plots, Bokeh excels at creating
interactive plots and dashboards. It can handle large datasets and streaming data,
making it suitable for real-time applications.

**Key Features of Bokeh:**

* **Interactivity:** Built-in support for zooming, panning, hovering, and other interactive tools.
* **Web-Focused:** Generates HTML and JavaScript, making it easy to embed plots in web pages.
* **High Performance:** Can handle large datasets efficiently.
* **Versatility:** Supports a wide range of plot types (lines, bars, scatter plots, etc.).

<hr style="border: 2px solid black;">


**Documentation:**

For comprehensive documentation, please refer to the official Bokeh website: [https://docs.bokeh.org/en/latest/](https://docs.bokeh.org/en/latest/)


<hr style="border: 2px solid black;">


**Lab Exercise:**

Your task is to recreate the time series analysis lab we previously conducted using Pandas,
Matplotlib, and Seaborn, but this time, utilize the Bokeh library for visualization.
This will involve:

1.  Loading and preprocessing the "AirPassengersDates.csv" dataset.
2.  Creating interactive Bokeh plots for:
    * Time series line plots
    * Bar plots of aggregated data
    * Visualizing mean and standard deviation
    * Outlier detection
    * Resampling (upsampling and downsampling)
    * Lag analysis
    * Autocorrelation

Pay close attention to Bokeh's features for interactivity (tools, hover effects) and
its handling of data sources. Aim to replicate the insights and visualizations
from the previous lab while leveraging Bokeh's strengths.

Good luck!
<hr style="border: 2px solid black;">

In [5]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, Band
from bokeh.layouts import column
from statsmodels.tsa.stattools import acf

output_notebook()

# === 1. Chargement et prétraitement ===
df = pd.read_csv("AirPassengersDates.csv")
df['Date'] = pd.to_datetime(df['Date'])
df.rename(columns={"#Passengers": "Passengers"}, inplace=True)
df['Passengers'] = pd.to_numeric(df['Passengers'], errors='coerce')
df.dropna(inplace=True)
df.set_index('Date', inplace=True)
source = ColumnDataSource(df)

# === 2. Courbe de séries temporelles ===
p1 = figure(title="Passagers au fil du temps", x_axis_type='datetime', width=800, height=300, tools="pan,wheel_zoom,box_zoom,reset,save")
p1.line('Date', 'Passengers', source=source, line_width=2)
hover1 = HoverTool(tooltips=[("Date", "@Date{%F}"), ("Passagers", "@Passengers")], formatters={"@Date": "datetime"})
p1.add_tools(hover1)

# === 3. Barres annuelles agrégées (corrigé) ===
df['Year'] = df.index.year
yearly = df.groupby('Year')['Passengers'].sum().reset_index()
yearly['Year'] = yearly['Year'].astype(str)  # Convertir en string pour Bokeh
source_yearly = ColumnDataSource(yearly)

p2 = figure(title="Total annuel de passagers", x_range=yearly['Year'].tolist(), width=800, height=300, tools="hover")
p2.vbar(x='Year', top='Passengers', width=0.7, source=source_yearly)
p2.xaxis.axis_label = "Année"
p2.yaxis.axis_label = "Passagers"

# === 4. Moyenne et écart-type ===
df['rolling_mean'] = df['Passengers'].rolling(12).mean()
df['rolling_std'] = df['Passengers'].rolling(12).std()
source_stats = ColumnDataSource(df)

p3 = figure(title="Moyenne mobile ± écart-type", x_axis_type='datetime', width=800, height=300)
p3.line('Date', 'rolling_mean', source=source_stats, color='green', legend_label="Moyenne mobile (12 mois)")
band = Band(base='Date', lower='rolling_mean', upper='rolling_mean+rolling_std', source=source_stats,
            level='underlay', fill_alpha=0.2, line_width=1, line_color='gray')
p3.add_layout(band)
p3.legend.location = "top_left"

# === 5. Détection des outliers ===
mean = df['Passengers'].mean()
std = df['Passengers'].std()
df['outlier'] = (df['Passengers'] > mean + 2*std) | (df['Passengers'] < mean - 2*std)
df['color'] = np.where(df['outlier'], "red", "blue")
source_outliers = ColumnDataSource(df)

p4 = figure(title="Valeurs aberrantes", x_axis_type='datetime', width=800, height=300)
p4.circle('Date', 'Passengers', source=source_outliers, size=6, color='color')

# === 6. Resampling ===
# Downsampling (annuel)
downsampled = df['Passengers'].resample('A').mean()
source_down = ColumnDataSource(data={'Date': downsampled.index, 'Passengers': downsampled.values})

p5 = figure(title="Downsampling - Moyenne annuelle", x_axis_type='datetime', width=800, height=300)
p5.line('Date', 'Passengers', source=source_down, line_width=2, color='orange')

# Upsampling (hebdomadaire)
upsampled = df['Passengers'].resample('W').mean().interpolate()
source_up = ColumnDataSource(data={'Date': upsampled.index, 'Passengers': upsampled.values})

p6 = figure(title="Upsampling - Interpolation hebdomadaire", x_axis_type='datetime', width=800, height=300)
p6.line('Date', 'Passengers', source=source_up, line_width=2, color='purple')

# === 7. Analyse de décalage (lag) ===
df['Lag1'] = df['Passengers'].shift(1)
lag_df = df.dropna()
source_lag = ColumnDataSource(lag_df)

p7 = figure(title="Analyse de décalage (Lag 1)", width=600, height=400)
p7.circle('Lag1', 'Passengers', source=source_lag, size=6)
p7.xaxis.axis_label = "Passagers (t-1)"
p7.yaxis.axis_label = "Passagers (t)"

# === 8. Autocorrélation ===
acf_values = acf(df['Passengers'].dropna(), nlags=24)
lags = list(range(len(acf_values)))

p8 = figure(title="Autocorrélation (jusqu’à 24 mois)", width=800, height=300)
p8.vbar(x=lags, top=acf_values, width=0.5)
p8.xaxis.axis_label = "Décalage (mois)"
p8.yaxis.axis_label = "Corrélation"

# === Affichage final ===
show(column(p1, p2, p3, p4, p5, p6, p7, p8))


Loading BokehJS ...

C:\Users\gmtech\AppData\Local\Temp\ipykernel_5816\820733530.py:60: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  downsampled = df['Passengers'].resample('A').mean()
